# CS202: Compiler Construction

## In-class Exercises, Week of 01/23/2023

----

Online diff checker: diffchecker.com

# Part 1: Lvar

## Question 1

Write an interpreter `eval_lvar` for the `Lvar` language. Reference the grammar: Figure 2.2 (page 14) in the textbook, and our simplified AST in the [Assignment 2 Description](https://github.com/jnear/cs202-compiler-construction/blob/master/homework/hw2.md).

In [1]:
from typing import Dict
from cs202_support.python import *

# Structure for an interpreter 
# 1. Create a new nested functiion for each ledt-hand side of the grammar
# 2. In each nested function match the argument
# 3. add a match case for each right-hand side of the grammar

# Notes, variables need something stored in them to use them
# Environment will map variables tro values
# env: vars -> val
# env: {x:5} <- we will be using python dictionaries for this

# Compiler will work slightly differently as the environment might need to map
# variables to expressions not necessarily values

def eval_lvar(program: Program):
    
#     op ::= "add"
#     Expr ::= Var(x) | Constant(n) | Prim(op, [Expr])
#     Stmt ::= Assign(x, Expr) | Print(Expr)
#     LVar ::= Program([Stmt])
    
    # Don't necessary need to def within eval_lvar but its not really used anywhere else
    # so we keep it encapsulated
    def eval_stmt(s: Stmt, env: Dict[str, int]):
        match s:
            case Assign(x, e):
                env[x]=eval_expr(e, env)
            case Print(e):
                v = eval_expr(e, env)
                print(v)
    
    def eval_expr(e: Expr, env: Dict[str, int]) -> int: # This dictionary takes strings to ints
        match e:
            case Var(x):
                return env[x]
            case Constant(n): # value of a number is just the number
                return n
            case Prim('add', [e1, e2]): # evaluate each expression and then add them together
                val1 = eval_expr(e1, env)
                val2 = eval_expr(e2, env)
                return val1+val2

    env = {}
    for s in program.stmts:
        eval_stmt(s, env)

In [2]:
# TEST CASE
program = """
x = 5
y = 6
print(x + y)"""

eval_lvar(parse(program))

11


----
# PART 2: Remove Complex Operands

## Question 2

Consider this translation of an expression to assembly language. What is wrong with it?

In [3]:
python = """
x = 1 + 2 + 3
"""

asm = """
movq $2, %rax
addq $1, (addq $3, %rax)
"""

Assembly language doesn't do nested instructions. Arguments need to be "atomic".


atomic:  
- Register
- Memory Location
- Constant

## Question 3

Which AST nodes in the language `Lvar` are **atomic**?

- Var(x)
- Constant(n)

Prim is not atomic as it has sub-expressions

## Question 4

Why do we need this pass? What is the form of its output?

This pass un-nests nested expressions because you can't have nested expressions in x86 Assembly language.

The form of the output:
 - The output of this pass will have primitives with only atomic arguments

This format is called A-normal (or monatic normal form in txtbook)

## Question 5

Convert the program from earlier into A-normal form.

In [4]:
python = """
x = 1 + 2 + 3
"""

In [5]:
python_anf = """
x = 1 + 2
x = x + 3 <--- my guess
"""

python_anf = """
tmp1 = 1 + 2
x = tmp1 + 3
"""

## Question 6

Describe a recursive procedure to perform the *remove-complex-opera* pass. Reference section 2.4 in the textbook.

Our structure will follow the stucture of the grammar:

- rco_exp compiles an expression
    - Constants and variables just return it
    - For a Prim expression, that might have expressions that are not atomic
        which we need to make atomic.
        - For each arg to the prim, create a new tmp variable (if needed) and bind it to the result of compiling the argument expression.
        - We can store new bindings in an environment which maps strings to expressions str -> Expr
- rco_stmt compiles a statement
    - Assign(x,e): call rco_exp on e
    - Print(e): call rco_exp on e
    - Challenge: what about bindings?
- rco_stmts compiles a list of statements
    - For each stmt
        - call rco stmt on each stmt
        - turn the bindings that were created into assignment statements


----
# Part 3: Select-instructions

The select-instructions pass transforms a sequence of statements into X86 assembly instructions.

## Question 7

Convert the following `Lvar` code into a psuedo-x86 assembly program.

```
Module([Assign([Name("y")], Constant(5)),
        Assign([Name("x")], Name("y")),
        Expr(Call(Name("print"), [Name("x")]))
])
```

SKIP FOR NOW (Do it Monday)

## Question 8

Describe the structure of select-instructions.

SKIP FOR NOW (Do it Monday)